# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f402083d2b0>
├── 'a' --> tensor([[-0.1963, -0.3187,  0.4260],
│                   [ 0.4080, -0.3906, -1.4545]])
└── 'x' --> <FastTreeValue 0x7f40228571f0>
    └── 'c' --> tensor([[-0.1158,  0.3914,  0.2431,  0.8812],
                        [ 1.5812,  0.0965, -0.7085,  0.7487],
                        [ 0.7772,  0.4698, -1.6489, -0.8629]])

In [4]:
t.a

tensor([[-0.1963, -0.3187,  0.4260],
        [ 0.4080, -0.3906, -1.4545]])

In [5]:
%timeit t.a

61.2 ns ± 0.0527 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f402083d2b0>
├── 'a' --> tensor([[ 1.1681, -0.0847, -1.1002],
│                   [ 0.3769,  0.4170, -1.0248]])
└── 'x' --> <FastTreeValue 0x7f40228571f0>
    └── 'c' --> tensor([[-0.1158,  0.3914,  0.2431,  0.8812],
                        [ 1.5812,  0.0965, -0.7085,  0.7487],
                        [ 0.7772,  0.4698, -1.6489, -0.8629]])

In [7]:
%timeit t.a = new_value

65.7 ns ± 0.0349 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1963, -0.3187,  0.4260],
               [ 0.4080, -0.3906, -1.4545]]),
    x: Batch(
           c: tensor([[-0.1158,  0.3914,  0.2431,  0.8812],
                      [ 1.5812,  0.0965, -0.7085,  0.7487],
                      [ 0.7772,  0.4698, -1.6489, -0.8629]]),
       ),
)

In [10]:
b.a

tensor([[-0.1963, -0.3187,  0.4260],
        [ 0.4080, -0.3906, -1.4545]])

In [11]:
%timeit b.a

55.7 ns ± 0.0396 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5621,  0.3985,  0.1620],
               [ 0.0837, -0.5106,  2.1501]]),
    x: Batch(
           c: tensor([[-0.1158,  0.3914,  0.2431,  0.8812],
                      [ 1.5812,  0.0965, -0.7085,  0.7487],
                      [ 0.7772,  0.4698, -1.6489, -0.8629]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.179 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

880 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 36.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 576 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3f52a196d0>
├── 'a' --> tensor([[[-0.1963, -0.3187,  0.4260],
│                    [ 0.4080, -0.3906, -1.4545]],
│           
│                   [[-0.1963, -0.3187,  0.4260],
│                    [ 0.4080, -0.3906, -1.4545]],
│           
│                   [[-0.1963, -0.3187,  0.4260],
│                    [ 0.4080, -0.3906, -1.4545]],
│           
│                   [[-0.1963, -0.3187,  0.4260],
│                    [ 0.4080, -0.3906, -1.4545]],
│           
│                   [[-0.1963, -0.3187,  0.4260],
│                    [ 0.4080, -0.3906, -1.4545]],
│           
│                   [[-0.1963, -0.3187,  0.4260],
│                    [ 0.4080, -0.3906, -1.4545]],
│           
│                   [[-0.1963, -0.3187,  0.4260],
│                    [ 0.4080, -0.3906, -1.4545]],
│           
│                   [[-0.1963, -0.3187,  0.4260],
│                    [ 0.4080, -0.3906, -1.4545]]])
└── 'x' --> <FastTreeValue 0x7f3f52a19250>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 44.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3f5ac3ac70>
├── 'a' --> tensor([[-0.1963, -0.3187,  0.4260],
│                   [ 0.4080, -0.3906, -1.4545],
│                   [-0.1963, -0.3187,  0.4260],
│                   [ 0.4080, -0.3906, -1.4545],
│                   [-0.1963, -0.3187,  0.4260],
│                   [ 0.4080, -0.3906, -1.4545],
│                   [-0.1963, -0.3187,  0.4260],
│                   [ 0.4080, -0.3906, -1.4545],
│                   [-0.1963, -0.3187,  0.4260],
│                   [ 0.4080, -0.3906, -1.4545],
│                   [-0.1963, -0.3187,  0.4260],
│                   [ 0.4080, -0.3906, -1.4545],
│                   [-0.1963, -0.3187,  0.4260],
│                   [ 0.4080, -0.3906, -1.4545],
│                   [-0.1963, -0.3187,  0.4260],
│                   [ 0.4080, -0.3906, -1.4545]])
└── 'x' --> <FastTreeValue 0x7f40207d8100>
    └── 'c' --> tensor([[-0.1158,  0.3914,  0.2431,  0.8812],
                        [ 1.5812,  0.0965, -0.7085,  0.7487],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 94.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

70.3 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1963, -0.3187,  0.4260],
                [ 0.4080, -0.3906, -1.4545]],
       
               [[-0.1963, -0.3187,  0.4260],
                [ 0.4080, -0.3906, -1.4545]],
       
               [[-0.1963, -0.3187,  0.4260],
                [ 0.4080, -0.3906, -1.4545]],
       
               [[-0.1963, -0.3187,  0.4260],
                [ 0.4080, -0.3906, -1.4545]],
       
               [[-0.1963, -0.3187,  0.4260],
                [ 0.4080, -0.3906, -1.4545]],
       
               [[-0.1963, -0.3187,  0.4260],
                [ 0.4080, -0.3906, -1.4545]],
       
               [[-0.1963, -0.3187,  0.4260],
                [ 0.4080, -0.3906, -1.4545]],
       
               [[-0.1963, -0.3187,  0.4260],
                [ 0.4080, -0.3906, -1.4545]]]),
    x: Batch(
           c: tensor([[[-0.1158,  0.3914,  0.2431,  0.8812],
                       [ 1.5812,  0.0965, -0.7085,  0.7487],
                       [ 0.7772,  0.4698, -1.6489, -0.8629]],
         

In [26]:
%timeit Batch.stack(batches)

80.1 µs ± 167 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1963, -0.3187,  0.4260],
               [ 0.4080, -0.3906, -1.4545],
               [-0.1963, -0.3187,  0.4260],
               [ 0.4080, -0.3906, -1.4545],
               [-0.1963, -0.3187,  0.4260],
               [ 0.4080, -0.3906, -1.4545],
               [-0.1963, -0.3187,  0.4260],
               [ 0.4080, -0.3906, -1.4545],
               [-0.1963, -0.3187,  0.4260],
               [ 0.4080, -0.3906, -1.4545],
               [-0.1963, -0.3187,  0.4260],
               [ 0.4080, -0.3906, -1.4545],
               [-0.1963, -0.3187,  0.4260],
               [ 0.4080, -0.3906, -1.4545],
               [-0.1963, -0.3187,  0.4260],
               [ 0.4080, -0.3906, -1.4545]]),
    x: Batch(
           c: tensor([[-0.1158,  0.3914,  0.2431,  0.8812],
                      [ 1.5812,  0.0965, -0.7085,  0.7487],
                      [ 0.7772,  0.4698, -1.6489, -0.8629],
                      [-0.1158,  0.3914,  0.2431,  0.8812],
                      [ 1.5812,  

In [28]:
%timeit Batch.cat(batches)

152 µs ± 312 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 6.24 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
